In [0]:
import sys
if "pyodide" in sys.modules:
    import piplite
    await piplite.install('opvious>=0.14.0')

# Modeling

We start with a few common model components.

In [2]:
import opvious.modeling as om

class Common(om.Model):
    horizon = om.Parameter.natural()  # Number of days to schedule for
    doctors = om.Dimension(name="I")  # Set of doctors
    days = om.interval(1, horizon(), name="T")  # Set of days
    shifts = om.Dimension(name="K")  # Set of shifts
    assigned = om.Variable.indicator(doctors, days, shifts)  # Shift assignment
    
    @om.constraint
    def at_most_one_shift_per_day(self):
        for i, t in self.doctors * self.days:
            yield om.total(self.assigned(i, t, k) for k in self.shifts) <= 1
    
common = Common()
common.specification().source()


<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">Common</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^p_\mathrm{horizon}&: h \in \mathbb{N} \\
  \S^d_\mathrm{doctors}&: I \\
  \S^a&: T \doteq \{ 1 \ldots h \} \\
  \S^d_\mathrm{shifts}&: K \\
  \S^v_\mathrm{assigned}&: \alpha \in \{0, 1\}^{I \times T \times K} \\
  \S^c_\mathrm{atMostOneShiftPerDay}&: \forall i \in I, t \in T, \sum_{k \in K} \alpha_{i,t,k} \leq 1 \\
\end{align*}
$$
</div>
</details>


## Consecutive changes

One way to model switches is to only count changes when a doctor is assigned different shifts on _consecutive_ days as a switch.

In [3]:
class SwitchOnConsecutiveShiftChanges(om.Model):
    def __init__(self):
        super().__init__(dependencies=[common])
        self.switched = om.Variable.indicator(common.doctors, common.days)
        
    def assigned_to_other(self, i, t, k):  # 1 if i assigned to a shift different than k on t, 0 otherwise
        return om.total(common.assigned(i, t, kk) for kk in common.shifts if kk != k)
        
    @om.constraint
    def consecutive_shift_change_forces_switch(self):
        for i, t, k in common.assigned.space():
            if t > 1:
                yield self.switched(i, t) >= common.assigned(i, t, k) + self.assigned_to_other(i, t-1, k) - 1
                    
SwitchOnConsecutiveShiftChanges().specification().source()


<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">SwitchOnConsecutiveShiftChanges</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^v_\mathrm{switched}&: \sigma \in \{0, 1\}^{I \times T} \\
  \S^c_\mathrm{consecutiveShiftChangeForcesSwitch}&: \forall i \in I, t \in T, k \in K \mid t > 1, \sigma_{i,t} \geq \alpha_{i,t,k} + \sum_{k' \in K \mid k' \neq k} \alpha_{i,t - 1,k'} - 1 \\
\end{align*}
$$
</div>
</details>


## All changes

Yet another way to model this is to count all shift changes as switches, including when a doctor starts or ends a shift. In this case, the switch variable is equivalent to the magnitude (absolute value) of assignment changes.

In [4]:
class SwitchOnAllShiftChanges(om.Model):
    def __init__(self):
        super().__init__(dependencies=[common])
            
    @om.fragments.magnitude_variable(*common.assigned.quantifiables(), projection=0b11)
    def switched(self, i, t, k):  # 1 if a different switch started, 0 if same shift, -1 if shift ended
        return common.assigned(i, t, k) - om.switch((t > 1, common.assigned(i, t-1, k)), 0)
                    
SwitchOnAllShiftChanges().specification().source()


<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">SwitchOnAllShiftChanges</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^v_\mathrm{switched}&: \sigma \in \mathbb{R}_+^{I \times T} \\
  \S^c_\mathrm{switchedLowerBounds}&: \forall i \in I, t \in T, k \in K, {-\sigma_{i,t}} \leq \alpha_{i,t,k} - \begin{cases} \alpha_{i,t - 1,k} \mid t > 1, \\ 0 \end{cases} \\
  \S^c_\mathrm{switchedUpperBounds}&: \forall i \in I, t \in T, k \in K, \sigma_{i,t} \geq \alpha_{i,t,k} - \begin{cases} \alpha_{i,t - 1,k} \mid t > 1, \\ 0 \end{cases} \\
\end{align*}
$$
</div>
</details>


# Solution comparison

In this section we compare the two approaches on a small example.

Our overall objective will be to minimize the total number of switches while ensuring that there is at least one doctor per shift. We also introduce a parameter allowing doctors to mark themselves as unavailable for a given shift.

In [5]:
class Scheduling(om.Model):
    unavailable = om.Parameter.indicator(common.assigned.quantifiables())
    
    def __init__(self, sm):
        super().__init__([sm])
        self._sm = sm
        
    @om.objective
    def minimize_switches(self):
        return om.total(self._sm.switched(i, t) for i, t in common.doctors * common.days)
    
    @om.constraint
    def at_least_one_doctor_per_shift(self):
        for t, k in common.days * common.shifts:
            yield om.total(common.assigned(i, t, k) >= 1 for i in common.doctors)
            
    @om.constraint
    def only_assigned_when_available(self):
        for i, t, k in common.assigned.space():
            if self.unavailable(i, t, k):
                yield common.assigned(i, t, k) == 0
    
Scheduling(SwitchOnAllShiftChanges()).specification().source()


<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">Scheduling</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^p_\mathrm{unavailable}&: u \in \{0, 1\}^{I \times T \times K} \\
  \S^o_\mathrm{minimizeSwitches}&: \min \sum_{i \in I, t \in T} \sigma_{i,t} \\
  \S^c_\mathrm{atLeastOneDoctorPerShift}&: \forall t \in T, k \in K, \sum_{i \in I} \alpha_{i,t,k} \geq 1 \\
  \S^c_\mathrm{onlyAssignedWhenAvailable}&: \forall i \in I, t \in T, k \in K \mid u_{i,t,k} \neq 0, \alpha_{i,t,k} = 0 \\
\end{align*}
$$
</div>
</details>


We now just need to create a small wrapper which will trigger a solve on toy data and pretty-print the optimal schedule.

In [9]:
import opvious

horizon = 14
shifts = ['early', 'midday', 'night']
unavailabilities = {
    'ann': [(1, 'midday'), (8, 'early'), (8, 'night')],
    'bob': [(2, 'night'), (14, 'midday')],
    'cat': [(1, 'early'), (10, 'midday'), (10, 'night')],
    'dan': [(5, 'midday')],
}

async def find_optimal_schedule(shifts_model):
    """Pretty-prints an optimal assignment schedule"""
    client = opvious.Client.default()
    res = await client.run_solve(
        specification=Scheduling(shifts_model).specification(),
        parameters={
            'horizon': horizon,
            'unavailable': [(d, t, k) for d, arr in unavailabilities.items() for t, k in arr]
        },
        dimensions={
            'doctors': unavailabilities.keys(),
            'shifts': shifts,
        },
        assert_feasible=True,
    )
    df = res.outputs.variable('assigned')  # Flat assignment dataframe
    return (
        df.reset_index()
            .drop('value', axis=1)
            .set_axis(['doctor', 'day', 'shift'], axis=1)
            .pivot(index=['day'], columns=['doctor'], values=['shift'])
            .fillna('')
    )

Running it using the first option, we can see that the doctors change shifts often, but never consecutively. This makes sense since the model isn't penalized for shifts that happen on either side of a break.

In [7]:
await find_optimal_schedule(SwitchOnConsecutiveShiftChanges())

shift                      
doctor     ann    bob     cat    dan
day                                 
1        night         midday  early
2        night  early  midday       
3        night  early  midday       
4        night  early  midday       
5        night  early  midday       
6               early  midday  night
7               early  midday  night
8               early  midday  night
9               early  midday  night
10      midday  early          night
11              early  midday  night
12              early  midday  night
13      midday  early          night
14      midday  early          night

If we use the second option, shift assignments are sticky: the model only changes a doctor's assignment if the doctor is unavailable. (In practice, we could add a constraint which ensures that doctors have a minimum number of days off within a rolling window of days.)

In [8]:
await find_optimal_schedule(SwitchOnAllShiftChanges())

shift                      
doctor     ann    bob     cat    dan
day                                 
1               early  midday  night
2               early  midday  night
3       midday  early          night
4       midday  early          night
5       midday  early          night
6       midday  early          night
7       midday  early          night
8       midday  early          night
9       midday  early          night
10      midday  early          night
11      midday  early          night
12      midday  early          night
13      midday  early          night
14      midday  early          night